 # Lab 2: 번역 기법 비교



 Zero-shot, Few-shot, 용어집 적용 번역 기법을 비교하고 품질 차이를 확인합니다.



 ## 학습 목표



 - Zero-shot, Few-shot, 용어집 적용 번역 기법 비교

 - 각 기법별 품질 차이 직접 확인

 - 최적의 번역 프롬프트 구성 방법 이해



 ## 워크샵 전체 흐름



 ```

 Lab 1: 환경 설정 ✅

 ↓

 Lab 2: 번역 기법 비교 (현재)

 ↓

 Lab 3: 품질 평가 - 충실도

 ↓

 Lab 4: 품질 평가 - 용어 일관성

 ↓

 Lab 5: 품질 평가 - 문화/톤

 ↓

 Lab 6: 피드백 기반 재번역

 ↓

 Lab 7: 종합 분석

 ```

 ---

 ## 2.1 환경 설정 및 Lab 1 결과 로드

In [1]:
import json
import os
from datetime import datetime

import boto3
import pandas as pd


In [3]:
# 설정
REGION = "us-west-2"
OUTPUT_DIR = "lab_outputs"

# Bedrock 클라이언트 생성
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION
)

print(f"Bedrock 클라이언트 초기화 완료 (리전: {REGION})")


Bedrock 클라이언트 초기화 완료 (리전: us-west-2)


In [4]:
# 모델 설정
CLAUDE_MODELS = {
    "sonnet": {
        "id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name": "Claude Sonnet 4",
    },
    "haiku": {
        "id": "us.anthropic.claude-haiku-4-20250514-v1:0",
        "name": "Claude Haiku 4",
    }
}

DEFAULT_MODEL = "sonnet"
MODEL_ID = CLAUDE_MODELS[DEFAULT_MODEL]["id"]

print(f"사용 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")


사용 모델: Claude Sonnet 4


In [5]:
# Lab 1 결과 로드
def load_results(filename: str) -> dict:
    """저장된 JSON 파일을 로드합니다."""
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"✓ 로드 완료: {filepath}")
    return data

workshop_config = load_results("workshop_config.json")
faq_items = workshop_config["faq_items"]

print(f"\n워크샵 FAQ 항목 수: {len(faq_items)}")
for item in faq_items:
    print(f"  - {item['id']}")


✓ 로드 완료: lab_outputs/workshop_config.json

워크샵 FAQ 항목 수: 6
  - IDS_FAQ_SC_MAIN_HEADER_02
  - IDS_FAQ_SC_MAIN_HEADER_03
  - IDS_FAQ_SC_ACCESSING_TEXT
  - IDS_FAQ_SC_MAIN_HEADER_13
  - IDS_FAQ_GO_DEVICE_NOTES_UL_01
  - IDS_FAQ_GO_MAIN_HEADER_27


In [6]:
# 결과 저장 함수
def save_results(data: dict, filename: str) -> str:
    """결과를 JSON 파일로 저장합니다."""
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    data["_metadata"] = {
        "saved_at": datetime.now().isoformat(),
        "model": CLAUDE_MODELS[DEFAULT_MODEL]["name"]
    }
    
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f"✓ 저장 완료: {filepath}")
    return filepath


 ---

 ## 2.2 Claude API 호출 함수

In [7]:
def call_claude(prompt: str, model_id: str = MODEL_ID, max_tokens: int = 2048) -> str:
    """Claude 모델을 호출하여 응답을 반환합니다."""
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType="application/json",
        accept="application/json"
    )
    
    response_body = json.loads(response["body"].read())
    return response_body["content"][0]["text"]


 ---

 ## 2.3 세 가지 번역 기법 소개



 ### Zero-shot 번역

 - 예시 없이 바로 번역 지시

 - 가장 간단하지만 품질이 일정하지 않을 수 있음



 ### Few-shot 번역

 - 좋은 번역 예시 3-5개를 프롬프트에 포함

 - 예시를 통해 원하는 스타일과 품질을 학습



 ### 용어집 적용 번역

 - 필수 용어 매핑을 프롬프트에 주입

 - 브랜드명, 기술 용어의 일관성 확보

 ---

 ## 2.4 Part 1: Zero-shot 번역



 예시 없이 기본 프롬프트로 번역합니다.

In [8]:
# Zero-shot 번역 프롬프트 템플릿
ZERO_SHOT_PROMPT = """다음 한국어 텍스트를 영어로 번역해주세요.

원문: {source}

번역 결과만 출력하세요. 다른 설명은 필요 없습니다."""


In [9]:
# Zero-shot 번역 실행
zero_shot_results = []

print("Zero-shot 번역 실행 중...")
print("=" * 80)

for item in faq_items:
    source = item["ko"]
    prompt = ZERO_SHOT_PROMPT.format(source=source)
    
    translation = call_claude(prompt)
    
    result = {
        "id": item["id"],
        "source": source,
        "source_lang": "ko",
        "target_lang": "en",
        "translation": translation.strip(),
        "method": "zero_shot",
        "reference_us": item["en_us"],
        "reference_gb": item["en_gb"],
        "timestamp": datetime.now().isoformat()
    }
    zero_shot_results.append(result)
    
    print(f"\n[{item['id']}]")
    print(f"원문: {source}")
    print(f"번역: {translation.strip()}")
    print(f"참조(US): {item['en_us']}")

print("\n" + "=" * 80)
print(f"Zero-shot 번역 완료: {len(zero_shot_results)}건")


Zero-shot 번역 실행 중...

[IDS_FAQ_SC_MAIN_HEADER_02]
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
번역: Do I need a Samsung account to use Samsung Cloud?
참조(US): Do I need a Samsung account to use Samsung Cloud?

[IDS_FAQ_SC_MAIN_HEADER_03]
원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?
번역: What happens when you remove a Samsung account from the device?
참조(US): What happens if I remove my Samsung account?

[IDS_FAQ_SC_ACCESSING_TEXT]
원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시면 사용자 정보 확인 후 차단 해제가 가능합니다.
번역: After accessing the Samsung Cloud webpage, please contact us through "Contact us" via "1:1 Inquiry" and we can lift the block after verifying your user information.
참조(US): After accessing the Samsung Cloud web page, contact us through the "1:1 inquiry" in "Contact us". We will unlock your Samsung Cloud after confirming your user information.

[IDS_FAQ_SC_MAIN_HEADER_13]
원문: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수 있는 방법이 있나요?
번역: I don't currently have a Galaxy device. Is there a way to check t

In [10]:
# Zero-shot 결과 저장
save_results({"translations": zero_shot_results}, "translations_zero_shot.json")


✓ 저장 완료: lab_outputs/translations_zero_shot.json


'lab_outputs/translations_zero_shot.json'

 ---

 ## 2.5 Part 2: Few-shot 번역



 좋은 번역 예시를 제공하여 품질을 높입니다.

In [15]:
# Few-shot 예시 정의 (US/GB 차이가 있는 실제 번역 예시)
FEW_SHOT_EXAMPLES = [
    {
        "ko": "삼성 계정을 삭제하면 어떻게 되나요?",
        "en": "What happens if I delete my Samsung account?"  # delete 사용, my 사용
    },
    {
        "ko": "클라우드 휴지통은 어디서 확인하나요?",
        "en": "Where can I find my Cloud Trash?"  # Trash 사용 (US 스타일)
    },
    {
        "ko": "동기화된 데이터를 확인할 수 있나요?",
        "en": "Can I check my synchronized data?"  # my 사용, synchronized 사용
    }
]


In [16]:
# Few-shot 번역 프롬프트 템플릿
def create_few_shot_prompt(source: str, examples: list) -> str:
    """Few-shot 프롬프트를 생성합니다."""
    
    examples_text = ""
    for i, ex in enumerate(examples, 1):
        examples_text += f"""
예시 {i}:
원문: {ex['ko']}
번역: {ex['en']}
"""
    
    prompt = f"""다음 한국어 텍스트를 영어로 번역해주세요.
아래 예시들의 스타일과 톤을 참고하세요.

{examples_text}

이제 다음 텍스트를 번역해주세요:

원문: {source}

번역 결과만 출력하세요. 다른 설명은 필요 없습니다."""
    
    return prompt


In [17]:
# Few-shot 번역 실행
few_shot_results = []

print("Few-shot 번역 실행 중...")
print("=" * 80)

for item in faq_items:
    source = item["ko"]
    prompt = create_few_shot_prompt(source, FEW_SHOT_EXAMPLES)
    
    translation = call_claude(prompt)
    
    result = {
        "id": item["id"],
        "source": source,
        "source_lang": "ko",
        "target_lang": "en",
        "translation": translation.strip(),
        "method": "few_shot",
        "reference_us": item["en_us"],
        "reference_gb": item["en_gb"],
        "timestamp": datetime.now().isoformat()
    }
    few_shot_results.append(result)
    
    print(f"\n[{item['id']}]")
    print(f"원문: {source}")
    print(f"번역: {translation.strip()}")
    print(f"참조(US): {item['en_us']}")

print("\n" + "=" * 80)
print(f"Few-shot 번역 완료: {len(few_shot_results)}건")


Few-shot 번역 실행 중...



[IDS_FAQ_SC_MAIN_HEADER_02]
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
번역: Do I need a Samsung account to use Samsung Cloud?
참조(US): Do I need a Samsung account to use Samsung Cloud?

[IDS_FAQ_SC_MAIN_HEADER_03]
원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?
번역: What happens if I remove my Samsung account from my device?
참조(US): What happens if I remove my Samsung account?

[IDS_FAQ_SC_ACCESSING_TEXT]
원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시면 사용자 정보 확인 후 차단 해제가 가능합니다.
번역: After accessing the Samsung Cloud webpage, you can contact us through "Contact us" and use "1-on-1 Inquiry" to submit your request. We can unblock your account after verifying your user information.
참조(US): After accessing the Samsung Cloud web page, contact us through the "1:1 inquiry" in "Contact us". We will unlock your Samsung Cloud after confirming your user information.

[IDS_FAQ_SC_MAIN_HEADER_13]
원문: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수 있는 방법이 있나요?
번역: I don't currently have a Galaxy device. Is there a way t

In [18]:
# Few-shot 결과 저장
save_results({"translations": few_shot_results}, "translations_few_shot.json")


✓ 저장 완료: lab_outputs/translations_few_shot.json


'lab_outputs/translations_few_shot.json'

In [20]:
print("Few-shot 예시 특징: 'my' 사용, 'Trash' 사용, 'delete/if' 패턴")
print("=" * 90)

for i, item in enumerate(faq_items):
    zero = zero_shot_results[i]["translation"]
    few = few_shot_results[i]["translation"]
    
    if zero != few:
        print(f"\n[{item['id']}]")
        print(f"Zero: {zero}")
        print(f"Few:  {few}")

print("\n" + "=" * 90)
print("→ Few-shot 예시의 스타일(my, Trash, if)이 번역에 반영됨")



Few-shot 예시 특징: 'my' 사용, 'Trash' 사용, 'delete/if' 패턴

[IDS_FAQ_SC_MAIN_HEADER_03]
Zero: What happens when you remove a Samsung account from the device?
Few:  What happens if I remove my Samsung account from my device?

[IDS_FAQ_SC_ACCESSING_TEXT]
Zero: After accessing the Samsung Cloud webpage, please contact us through "Contact us" via "1:1 Inquiry" and we can lift the block after verifying your user information.
Few:  After accessing the Samsung Cloud webpage, you can contact us through "Contact us" and use "1-on-1 Inquiry" to submit your request. We can unblock your account after verifying your user information.

[IDS_FAQ_SC_MAIN_HEADER_13]
Zero: I don't currently have a Galaxy device. Is there a way to check the data or information that was saved and synced to Samsung services?
Few:  I don't currently have a Galaxy device. Is there a way to check data or information that was saved and synchronized to Samsung services?

[IDS_FAQ_GO_DEVICE_NOTES_UL_01]
Zero: Support availability may v

 ---

 ## 2.6 Part 3: 용어집 적용 번역



 삼성클라우드 전용 용어집을 적용하여 일관성을 확보합니다.

In [21]:
# 삼성클라우드 용어집 정의
GLOSSARY = {
    # 브랜드명 (번역 금지)
    "Samsung Cloud": "Samsung Cloud",
    "Samsung account": "Samsung account",
    "Samsung Galaxy": "Galaxy",
    "Galaxy": "Galaxy",
    "OneDrive": "OneDrive",
    "Microsoft": "Microsoft",
    
    # 기술 용어 (일관된 번역)
    "동기화": "sync",
    "백업": "backup",
    "복원": "restore",
    "단말": "device",
    "저장공간": "storage",
    "휴지통": "Trash",  # US 스타일
    "설정": "Settings",
    "계정": "account",
    "연동": "integration",
    "암호화": "encryption",
    
    # UI 용어
    "더 보기": "More",
    "문의하기": "Contact us",
}


In [22]:
# 용어집 적용 번역 프롬프트 템플릿
def create_glossary_prompt(source: str, glossary: dict) -> str:
    """용어집을 적용한 프롬프트를 생성합니다."""
    
    glossary_text = "다음 용어집을 반드시 준수하세요:\n"
    for ko, en in glossary.items():
        glossary_text += f"  - {ko} → {en}\n"
    
    prompt = f"""다음 한국어 텍스트를 영어로 번역해주세요.

{glossary_text}

원문: {source}

번역 규칙:
1. 용어집에 있는 단어는 반드시 지정된 영어 표현을 사용하세요.
2. 브랜드명(Samsung Cloud, Samsung account, Galaxy, OneDrive)은 번역하지 마세요.
3. 자연스러운 영어 문장이 되도록 번역하세요.

번역 결과만 출력하세요. 다른 설명은 필요 없습니다."""
    
    return prompt


In [23]:
# 용어집 적용 번역 실행
glossary_results = []

print("용어집 적용 번역 실행 중...")
print("=" * 80)

for item in faq_items:
    source = item["ko"]
    prompt = create_glossary_prompt(source, GLOSSARY)
    
    translation = call_claude(prompt)
    
    result = {
        "id": item["id"],
        "source": source,
        "source_lang": "ko",
        "target_lang": "en",
        "translation": translation.strip(),
        "method": "glossary",
        "reference_us": item["en_us"],
        "reference_gb": item["en_gb"],
        "timestamp": datetime.now().isoformat()
    }
    glossary_results.append(result)
    
    print(f"\n[{item['id']}]")
    print(f"원문: {source}")
    print(f"번역: {translation.strip()}")
    print(f"참조(US): {item['en_us']}")

print("\n" + "=" * 80)
print(f"용어집 적용 번역 완료: {len(glossary_results)}건")


용어집 적용 번역 실행 중...

[IDS_FAQ_SC_MAIN_HEADER_02]
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
번역: Do I need a Samsung account to use Samsung Cloud?
참조(US): Do I need a Samsung account to use Samsung Cloud?

[IDS_FAQ_SC_MAIN_HEADER_03]
원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?
번역: What happens when you remove your Samsung account from the device?
참조(US): What happens if I remove my Samsung account?

[IDS_FAQ_SC_ACCESSING_TEXT]
원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시면 사용자 정보 확인 후 차단 해제가 가능합니다.
번역: After accessing the Samsung Cloud webpage, please contact us through "Contact us" → "1:1 inquiry" and we can unblock your account after verifying your user information.
참조(US): After accessing the Samsung Cloud web page, contact us through the "1:1 inquiry" in "Contact us". We will unlock your Samsung Cloud after confirming your user information.

[IDS_FAQ_SC_MAIN_HEADER_13]
원문: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수 있는 방법이 있나요?
번역: I don't currently have a Galaxy device. Is there a way to che

In [24]:
# 용어집 적용 결과 저장
save_results({
    "translations": glossary_results,
    "glossary": GLOSSARY
}, "translations_glossary.json")


✓ 저장 완료: lab_outputs/translations_glossary.json


'lab_outputs/translations_glossary.json'

 ---

 ## 2.7 Part 4: 세 기법 비교



 Zero-shot, Few-shot, 용어집 적용 번역 결과를 나란히 비교합니다.

In [25]:
# 비교 테이블 생성
print("=" * 100)
print("번역 기법 비교")
print("=" * 100)

for i, item in enumerate(faq_items):
    item_id = item["id"]
    
    zero_trans = zero_shot_results[i]["translation"]
    few_trans = few_shot_results[i]["translation"]
    glossary_trans = glossary_results[i]["translation"]
    
    print(f"\n[{item_id}]")
    print(f"원문: {item['ko']}")
    print("-" * 80)
    print(f"Zero-shot:  {zero_trans}")
    print(f"Few-shot:   {few_trans}")
    print(f"용어집:     {glossary_trans}")
    print("-" * 80)
    print(f"참조(US):   {item['en_us']}")
    print(f"참조(GB):   {item['en_gb']}")
    print("=" * 100)


번역 기법 비교

[IDS_FAQ_SC_MAIN_HEADER_02]
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
--------------------------------------------------------------------------------
Zero-shot:  Do I need a Samsung account to use Samsung Cloud?
Few-shot:   Do I need a Samsung account to use Samsung Cloud?
용어집:     Do I need a Samsung account to use Samsung Cloud?
--------------------------------------------------------------------------------
참조(US):   Do I need a Samsung account to use Samsung Cloud?
참조(GB):   Do I need my Samsung account to use Samsung Cloud?

[IDS_FAQ_SC_MAIN_HEADER_03]
원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?
--------------------------------------------------------------------------------
Zero-shot:  What happens when you remove a Samsung account from the device?
Few-shot:   What happens if I remove my Samsung account from my device?
용어집:     What happens when you remove your Samsung account from the device?
--------------------------------------------------------------------------------
참조(US):   What hap

In [26]:
# DataFrame으로 비교
comparison_data = []

for i, item in enumerate(faq_items):
    comparison_data.append({
        "id": item["id"],
        "source": item["ko"][:30] + "...",
        "zero_shot": zero_shot_results[i]["translation"][:50] + "...",
        "few_shot": few_shot_results[i]["translation"][:50] + "...",
        "glossary": glossary_results[i]["translation"][:50] + "...",
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n번역 결과 요약:")
comparison_df



번역 결과 요약:


,id,source,zero_shot,few_shot,glossary
0,IDS_FAQ_SC_MAIN_HEADER_02,삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?...,Do I need a Samsung account to use Samsung Clo...,Do I need a Samsung account to use Samsung Clo...,Do I need a Samsung account to use Samsung Clo...
1,IDS_FAQ_SC_MAIN_HEADER_03,삼성 계정을 단말에서 제거하면 어떻게 되나요?...,What happens when you remove a Samsung account...,What happens if I remove my Samsung account fr...,What happens when you remove your Samsung acco...
2,IDS_FAQ_SC_ACCESSING_TEXT,삼성 클라우드 웹페이지 접속 후 Contact us를 ...,"After accessing the Samsung Cloud webpage, ple...","After accessing the Samsung Cloud webpage, you...","After accessing the Samsung Cloud webpage, ple..."
3,IDS_FAQ_SC_MAIN_HEADER_13,갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 ...,I don't currently have a Galaxy device. Is the...,I don't currently have a Galaxy device. Is the...,I don't currently have a Galaxy device. Is the...
4,IDS_FAQ_GO_DEVICE_NOTES_UL_01,"특정 단말, 국가, 사업자에 따라 지원 여부는 다를 수...",Support availability may vary depending on the...,Support may vary depending on the specific dev...,Support may vary depending on the specific dev...
5,IDS_FAQ_GO_MAIN_HEADER_27,어디로 가면 클라우드 휴지통을 확인할 수 있나요?...,Where can I go to check the cloud recycle bin?...,Where can I go to check my Cloud Trash?...,Where can I go to check the cloud Trash?...


 ---

 ## 2.8 관찰 포인트



 각 기법별로 다음 사항을 관찰해보세요:



 ### Zero-shot

 - 기본적인 번역 품질은 어떤가요?

 - 용어 선택이 일관적인가요?



 ### Few-shot

 - 예시를 통해 스타일이 개선되었나요?

 - Zero-shot과 비교해서 어떤 차이가 있나요?



 ### 용어집 적용

 - 브랜드명이 올바르게 유지되었나요?

 - 기술 용어가 일관되게 번역되었나요?

 - "휴지통"이 "Trash"로 번역되었나요? (US 스타일 용어집 적용)

 ---

 ## 2.9 Lab 2 완료



 세 가지 번역 기법의 결과를 저장했습니다.

 다음 Lab에서 이 결과들의 품질을 평가합니다.

In [27]:
print("=" * 80)
print("Lab 2: 번역 기법 비교 완료")
print("=" * 80)

print("\n[저장된 파일]")
print(f"  - {OUTPUT_DIR}/translations_zero_shot.json")
print(f"  - {OUTPUT_DIR}/translations_few_shot.json")
print(f"  - {OUTPUT_DIR}/translations_glossary.json")

print("\n[번역 결과 요약]")
print(f"  - Zero-shot: {len(zero_shot_results)}건")
print(f"  - Few-shot: {len(few_shot_results)}건")
print(f"  - 용어집 적용: {len(glossary_results)}건")

print("\n[다음 단계]")
print("  → Lab 3: 품질 평가 - 충실도")
print("    번역 결과의 의미 보존, 정확성을 평가합니다.")

print("\n" + "=" * 80)


Lab 2: 번역 기법 비교 완료

[저장된 파일]
  - lab_outputs/translations_zero_shot.json
  - lab_outputs/translations_few_shot.json
  - lab_outputs/translations_glossary.json

[번역 결과 요약]
  - Zero-shot: 6건
  - Few-shot: 6건
  - 용어집 적용: 6건

[다음 단계]
  → Lab 3: 품질 평가 - 충실도
    번역 결과의 의미 보존, 정확성을 평가합니다.

